In [2]:
import os
import gurobipy as gp
import numpy as np
import json
import pandas as pd

## Benchmark single instance

In [ ]:
qubo_path = "../data/portfolio/QUBO/qubo_Ns10_Nt9_Nq2_K10_gamma1_zeta0.042_rho1.0.json"

with open(qubo_path, "r") as f:
    qubo_data = json.load(f)

J = qubo_data["J"]
h = qubo_data["h"]
offset = qubo_data.get("c", 0)

# Create a Gurobi model
model = gp.Model("QUBO_Model")
model.Params.TimeLimit = 600  # Set a time limit of 20 seconds

# Add binary variables
x = model.addVars(h.keys(), vtype=gp.GRB.BINARY, name="x")

# Set the objective function
obj = gp.quicksum(h[i] * x[i] for i in h.keys())
obj += gp.quicksum(J[k] * x[k.split(',')[0]] * x[k.split(',')[1]] for k in J.keys())

# Constrain x0 == 1
#model.addConstr(x['0'] == 1, "c0")

model.setObjective(obj, gp.GRB.MINIMIZE)

# Optimize the model
model.optimize()

# Give output bitstring as list
solution = [int(x[i].X) for i in h.keys()]
print("Optimal solution:", solution)

# Print cost with offset
cost = model.ObjVal + offset
print("Optimal cost (with offset):", cost)


Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Core(TM) i7-7820X CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  600

Academic license 2674347 - for non-commercial use only - registered to er___@kth.se
Optimize a model with 0 rows, 180 columns and 0 nonzeros
Model fingerprint: 0x4d321f98
Model has 2030 quadratic objective terms
Variable types: 0 continuous, 180 integer (180 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [2e-01, 4e-01]
  QObjective range [2e-03, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000
Found heuristic solution: objective -9.0836485
Found heuristic solution: objective -9.0905868
Presolve time: 0.01s
Presolved: 2030 rows, 2210 columns, 6090 nonzer

## Benchmark batch of instances

In [ ]:
time_limit = 60

graph_type = "SK"
num_vertices = 100
problem_dir = f"../data/MaxCut/{graph_type}/{num_vertices}v/QUBO/"
output_dir = f"../gurobi_results/MaxCut/{graph_type}/{num_vertices}v/"

costs = []
for idx in range(50):
    file_path = os.path.join(problem_dir, f"qubo_graph{idx}.json")
    with open(file_path, "r") as f:
        qubo_data = json.load(f)

    J = qubo_data["J"]
    h = qubo_data["h"]
    offset = qubo_data.get("c", 0)

    # Create a Gurobi model
    model = gp.Model("QUBO_Model")
    model.Params.TimeLimit = time_limit  # Set a time limit of 60 seconds

    # Add binary variables
    x = model.addVars(h.keys(), vtype=gp.GRB.BINARY, name="x")

    # Set the objective function
    obj = gp.quicksum(h[i] * x[i] for i in h.keys())
    obj += gp.quicksum(J[k] * x[k.split(',')[0]] * x[k.split(',')[1]] for k in J.keys())

    model.setObjective(obj, gp.GRB.MINIMIZE)

    # Optimize the model
    model.optimize()

    # Give output bitstring as list
    solution = [int(x[i].X) for i in h.keys()]
    #print("Optimal solution:", solution)

    # Print cost with offset
    cost = model.ObjVal + offset
    costs.append(cost)
    print(f"Optimal cost for {file_path} (with offset):", cost)

    # Save the model
# Save costs to a csv at benchmark_results/MaxCut/3Reg/50v/gurobi_costs.csv
df = pd.DataFrame(costs, columns=["cost"])
os.makedirs(output_dir, exist_ok=True)
df.to_csv(os.path.join(output_dir, "gurobi_costs.csv"), index=False)

Set parameter TimeLimit to value 60
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Core(TM) i7-7820X CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  60

Academic license 2674347 - for non-commercial use only - registered to er___@kth.se
Optimize a model with 0 rows, 100 columns and 0 nonzeros
Model fingerprint: 0xd2c1a9a8
Model has 4950 quadratic objective terms
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 2e+01]
  QObjective range [4e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000
Found heuristic solution: objective -409.0000000
Presolve time: 0.01s
Presolved: 0 rows, 100 columns, 0 nonzeros
Presolved model has 4950 quadratic objective terms
